# Model Trainer

This notebook consists of code for training a model on the training data created and exported in `Text Summarization` Notebook. The data is available in the `data/opinosis-training.csv`, or as WANDB artifact, on the following [link](https://wandb.ai/aleksandar1932/[NLP]%20lab-03%20%7C%20text-summarization/artifacts/dataset/opinosis-training).

The code in this notebook will serve as starting basis for creating WANDB Sweeps for hyperparamenter tuning.

In [9]:
import os
import ast
from uuid import uuid4

import numpy as np
import pandas as pd
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
import wandb
from wandb.keras import WandbCallback

from scripts.model import create_model
from scripts.loader import load_embeddings_pkl

WANDB_PROJECT_NAME = os.getenv("WANDB_PROJECT_NAME") or "[NLP] lab-03 | text-summarization"
LEN_VOCABULARY = int(os.getenv("LEN_VOCABULARY")) or 7188
EMBEDDINGS_PATH = os.getenv("EMBEDDINGS_PATH") or 'data/embedding_matrix_glove_50.pkl'

Check GPU availability:

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Load Training Data

In [3]:
embeddings = load_embeddings_pkl(EMBEDDINGS_PATH)
df = pd.read_csv('data/opinosis-training.csv')

padded_texts = np.array([ast.literal_eval(sample) for sample in df.texts.to_list()])
padded_summaries = np.array([ast.literal_eval(sample) for sample in df.summaries.to_list()])
next_words = np.array([ast.literal_eval(sample) for sample in df.next_words.to_list()])

max_texts_length = len(padded_texts[0])
max_summaries_length = len(padded_summaries[0])

# Train Model

In [14]:
model = create_model(max_texts_length, max_summaries_length, LEN_VOCABULARY, 50, embeddings)
run = wandb.init(project=WANDB_PROJECT_NAME, job_type="train_model")
model.compile(optimizer=Adam(lr=0.001), loss=categorical_crossentropy, metrics=['accuracy'])
model.summary()

Model: "Encoder-Decoder-2e3a618b-5e15-49c4-b508-4fbd64f31b34"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, 6124)]       0           []                               
                                                                                                  
 decoder_inputs (InputLayer)    [(None, 23)]         0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 6124, 50)     359400      ['encoder_inputs[0][0]']         
                                                                                                  
 embedding_3 (Embedding)        (None, 23, 50)       359400      ['decoder_inputs[0][0]']         
                                               

D:\envs\nlp-64\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [16]:
model.fit([np.array(padded_texts), np.array(padded_summaries)],
              np.array(next_words),
              validation_split=0.1,
              batch_size=64, epochs=15, verbose=1,
              callbacks=[WandbCallback()]
              )

Epoch 1/15
8/8 [==============================] - 6s 706ms/step - loss: 8.4231 - accuracy: 0.0819 - val_loss: 8.2792 - val_accuracy: 0.0943
Epoch 2/15
8/8 [==============================] - 5s 621ms/step - loss: 7.2091 - accuracy: 0.0840 - val_loss: 7.6542 - val_accuracy: 0.0943
Epoch 3/15
8/8 [==============================] - 5s 614ms/step - loss: 5.9997 - accuracy: 0.0840 - val_loss: 7.6783 - val_accuracy: 0.0943
Epoch 4/15
8/8 [==============================] - 5s 612ms/step - loss: 5.5154 - accuracy: 0.0840 - val_loss: 8.0143 - val_accuracy: 0.0943
Epoch 5/15
8/8 [==============================] - 5s 602ms/step - loss: 5.4116 - accuracy: 0.0840 - val_loss: 8.2817 - val_accuracy: 0.0943
Epoch 6/15
8/8 [==============================] - 5s 584ms/step - loss: 5.3725 - accuracy: 0.0840 - val_loss: 8.4202 - val_accuracy: 0.0943
Epoch 7/15
8/8 [==============================] - 5s 596ms/step - loss: 5.3540 - accuracy: 0.0840 - val_loss: 8.4976 - val_accuracy: 0.0943
Epoch 8/15
8/8 [====

In [11]:
model_name = f"models/opinosis_model-{uuid4()}.h5"
model.save(model_name)
wandb.save(model)
run.finish()